In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ssl

from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf 
from sklearn.metrics import  recall_score, accuracy_score, f1_score, precision_score
from sklearn.metrics import confusion_matrix

In [14]:
try:
    diabetes_data = pd.read_csv('diabetes_scale.txt', sep=' ', header=None)
    diabetes_data = diabetes_data.drop(columns=[9]) # dropping a column 9 which is not reuqired as all are NaN values
    diabetes_data
except:
    print("Error")

# Changing the column names as original
column_names = {
    0: 'Outcome',
    1: 'Pregnancies',
    2: 'Glucose',
    3: 'BloodPressure',
    4: 'SkinThickness',
    5: 'Insulin',
    6: 'BMI',
    7: 'DiabetesPedigreeFunction',
    8: 'Age',
}
diabetes_data.rename(columns = column_names, inplace = True)

# This is to remove the 1:, 2:, from the front of observations
for col in diabetes_data.columns:
    if col != 'label':
        diabetes_data[col] = diabetes_data[col].astype(str).str.replace(r'^\d+:', '', regex=True).astype(float)

diabetes_data['Outcome'] = diabetes_data['Outcome'].replace(-1, 0)
diabetes_data

,Outcome,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.0,-0.294118,0.487437,0.180328,-0.292929,-1.000000,0.001490,-0.531170,-0.033333
1,1.0,-0.882353,-0.145729,0.081967,-0.414141,-1.000000,-0.207153,-0.766866,-0.666667
2,0.0,-0.058824,0.839196,0.049180,-1.000000,-1.000000,-0.305514,-0.492741,-0.633333
3,1.0,-0.882353,-0.105528,0.081967,-0.535354,-0.777778,-0.162444,-0.923997,-1.000000
4,0.0,-1.000000,0.376884,-0.344262,-0.292929,-0.602837,0.284650,0.887276,-0.600000
...,...,...,...,...,...,...,...,...,...
763,1.0,0.176471,0.015075,0.245902,-0.030303,-0.574468,-0.019374,-0.920581,0.400000
764,1.0,-0.764706,0.226131,0.147541,-0.454545,-1.000000,0.096870,-0.776260,-0.800000
765,1.0,-0.411765,0.216080,0.180328,-0.535354,-0.735225,-0.219076,-0.857387,-0.700000
766,0.0,-0.882353,0.266332,-0.016393,-1.000000,-1.000000,-0.102832,-0.768574,-0.133333


In [10]:
diabetes_data.describe().T[['count', 'mean', 'std', 'min']]
# Summary of the dataframe

,count,mean,std,min
Outcome,768.0,0.651042,0.476951,0.0
Pregnancies,768.0,-0.547641,0.396421,-1.0
Glucose,768.0,0.215020,0.321333,-1.0
BloodPressure,768.0,0.131574,0.319896,-1.0
SkinThickness,768.0,-0.585122,0.322267,-1.0
Insulin,768.0,-0.810410,0.273036,-1.0
BMI,768.0,-0.047279,0.237041,-1.0
DiabetesPedigreeFunction,768.0,-0.662247,0.284229,-1.0
Age,759.0,-0.599166,0.388652,-1.0


In [11]:
# Count of missing values
diabetes_data.isna().sum()

Outcome                     0
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         9
dtype: int64

In [12]:
iterative_imputer = IterativeImputer()
diabetes_data_iterative_imputed = pd.DataFrame(iterative_imputer.fit_transform(diabetes_data), columns=[ 'Outcome', 'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'])

diabetes_data_iterative_imputed.isna().sum()

Outcome                     0
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64

In [6]:
X = diabetes_data_iterative_imputed.drop(columns = ['Outcome'])
y = diabetes_data_iterative_imputed['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=18, stratify=y)

In [7]:
# This is an implementation of baseline perceptron using tensorflow sequencial and passing 1
# in layers.Dense this shows that 1 perceptron is used for implementation

# The function takes two arguments that are activation function and epochs

def Perceptron_algorithm(activation='relu', learning_rate=0.01):
    # Default learning rate used is 0.01, and activation_function is relu
    model = keras.Sequential()

    # adding just one perceptron and specifying the activation function
    model.add(layers.Dense(1, input_shape=(8,), activation=activation))
    
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Creating a wrapper to use grid search CV because 
# tensorflow does not have one for fine tuning
model = KerasClassifier(build_fn=Perceptron_algorithm, verbose=0)
model

# Parameter grid
param_grid_search = {
    'activation': ['relu'],
    'learning_rate': [1e-4, 1e-3, 1e-2, 1e-1],
    'epochs': [10]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid_search, n_jobs=-1, cv=3, verbose=2)

# Fitting the model
grid_result = grid.fit(X_train, y_train)

grid_scores = grid.cv_results_

# Getting all the results
cv_results = pd.DataFrame(grid_scores)[['mean_fit_time', 'param_activation', 'param_learning_rate', 'mean_test_score']]
cv_results = cv_results.sort_values(by = "mean_test_score", ascending=False)
cv_results

C:\Users\a1896438\AppData\Local\Temp\ipykernel_26372\2848280008.py:20: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=Perceptron, verbose=0)


Fitting 3 folds for each of 4 candidates, totalling 12 fits


,mean_fit_time,param_activation,param_learning_rate,mean_test_score
2,0.630963,relu,0.0100,0.671460
1,0.731075,relu,0.0010,0.617950
0,0.794722,relu,0.0001,0.487816
3,0.722077,relu,0.1000,0.348811


In [8]:
cv_results.sort_values(by = 'mean_test_score', ascending=False)[:5]

,mean_fit_time,param_activation,param_learning_rate,mean_test_score
2,0.630963,relu,0.0100,0.671460
1,0.731075,relu,0.0010,0.617950
0,0.794722,relu,0.0001,0.487816
3,0.722077,relu,0.1000,0.348811


In [9]:
grid.best_params_

{'activation': 'relu', 'epochs': 10, 'learning_rate': 0.01}

In [10]:
best_indices = np.argsort(grid.cv_results_['mean_test_score'])[-5:]
predictions = []
for idx in best_indices:
    # This makes models for the first five best params
    params = grid.cv_results_['params'][idx]
    model = grid.estimator.set_params(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    predictions.append({
        'params': params,
        'predictions': preds
    })

# The output was later confined to best model after this cell

3/3 [==============================] - 0s 2ms/step


Best model for relu

In [11]:
best_model_ = grid_result.best_estimator_
preds = best_model_.predict(X_test)
actual = y_test
pd.DataFrame({
    'act_func': [grid_result.best_params_['activation']],
    'epochs': [grid_result.best_params_['epochs']],
    'learning_rate': [grid_result.best_params_['learning_rate']],
    'F1 Score': f1_score(actual, preds),
    'Accu': accuracy_score(actual, preds),
    'Recall': recall_score(actual, preds),
    'Precision': precision_score(actual, preds)
})

3/3 [==============================] - 0s 1ms/step


,activation_function,epochs,learning_rate,F1 Score,Accuracy,Recall,Precision
0,relu,10,0.01,0.787402,0.649351,1.0,0.649351


In [12]:
TN, FP, FN, TP = confusion_matrix(y_test, preds).ravel()
# formulas for sens and spec
sensitivity = TP / (TP + FN)  
specificity = TN / (TN + FP)
print(f"Sensitivity is : {sensitivity:.2f}")
print(f"Specificity is : {specificity:.2f}")

Sensitivity is : 1.00
Specificity is : 0.00


Using tanh

In [13]:
# Creating a wrapper to use grid search CV because 
# tensorflow does not have one for fine tuning
model = KerasClassifier(build_fn=Perceptron_algorithm, verbose=0)
model

# Parameter grid
param_grid_search = {
    'activation': ['tanh'],
    'learning_rate': [1e-4, 1e-3, 1e-2, 1e-1],
    'epochs': [10]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid_search, n_jobs=-1, cv=3, verbose=2)
# Fitting the model
grid_result = grid.fit(X_train, y_train)

grid_scores = grid.cv_results_

# Getting all the results
cv_results = pd.DataFrame(grid_scores)[['mean_fit_time', 'param_activation', 'param_learning_rate', 'mean_test_score']]
cv_results = cv_results.sort_values(by = "mean_test_score", ascending=False)
cv_results

Fitting 3 folds for each of 4 candidates, totalling 12 fits


C:\Users\a1896438\AppData\Local\Temp\ipykernel_26372\1305734628.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=Perceptron, verbose=0)


,mean_fit_time,param_activation,param_learning_rate,mean_test_score
3,0.816376,tanh,0.1000,0.746678
2,1.001367,tanh,0.0100,0.557507
1,0.724510,tanh,0.0010,0.557494
0,0.735846,tanh,0.0001,0.399555


Best models for tanh

In [14]:
best_model_ = grid_result.best_estimator_
preds = best_model_.predict(X_test)

pd.DataFrame({
    'activation_function': [grid_result.best_params_['activation']],
    'epochs': [grid_result.best_params_['epochs']],
    'learning_rate': [grid_result.best_params_['learning_rate']],
    'F1 Score': f1_score(actual, preds),
    'Accuracy': accuracy_score(actual, preds),
    'Recall': recall_score(actual, preds),
    'Precision': precision_score(actual, preds)
})

3/3 [==============================] - 0s 3ms/step


,activation_function,epochs,learning_rate,F1 Score,Accuracy,Recall,Precision
0,tanh,10,0.1,0.778761,0.675325,0.88,0.698413


In [15]:
TN, FP, FN, TP = confusion_matrix(y_test, preds).ravel()
# formulas for sens and spec
sensitivity = TP / (TP + FN)  
specificity = TN / (TN + FP)
print(f"Sensitivity is : {sensitivity:.2f}")
print(f"Specificity is : {specificity:.2f}")

Sensitivity is : 0.88
Specificity is : 0.30


In [16]:
grid.best_params_

{'activation': 'tanh', 'epochs': 10, 'learning_rate': 0.1}

SIGMOID

In [17]:
# Creating a wrapper to use grid search CV because 
# tensorflow does not have one for fine tuning
model = KerasClassifier(build_fn=Perceptron_algorithm, verbose=0)
model

# Parameter grid
param_grid_search = {
    'activation': ['sigmoid'],
    'learning_rate': [1e-4, 1e-3, 1e-2, 1e-1],
    'epochs': [10]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid_search, n_jobs=-1, cv=3, verbose=2)
# Fitting the model
grid_result = grid.fit(X_train, y_train)

grid_scores = grid.cv_results_

# Getting all the results
cv_results = pd.DataFrame(grid_scores)[['mean_fit_time', 'param_activation', 'param_learning_rate', 'mean_test_score']]
cv_results = cv_results.sort_values(by = "mean_test_score", ascending=False)
cv_results

# Addimg epochs to the dataframe
epochs = []
for i in grid_scores['params']:
    for j, k in i.items():
        if j == 'epochs':
            epochs.append(k)
epochs
cv_results['epochs'] = epochs
cv_results = cv_results[['mean_fit_time', 'param_activation', 'param_learning_rate',
        'epochs', 'mean_test_score']]
cv_results

Fitting 3 folds for each of 4 candidates, totalling 12 fits


C:\Users\a1896438\AppData\Local\Temp\ipykernel_26372\632029255.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=Perceptron, verbose=0)


,mean_fit_time,param_activation,param_learning_rate,epochs,mean_test_score
3,0.442906,sigmoid,0.1000,10,0.709129
2,0.505733,sigmoid,0.0100,10,0.643974
1,0.538870,sigmoid,0.0010,10,0.597553
0,0.552421,sigmoid,0.0001,10,0.392233


Best model for sigmoid

In [18]:
best_model_ = grid_result.best_estimator_
preds = best_model_.predict(X_test)

pd.DataFrame({
    'activation_function': [grid_result.best_params_['activation']],
    'epochs': [grid_result.best_params_['epochs']],
    'learning_rate': [grid_result.best_params_['learning_rate']],
    'F1 Score': f1_score(actual, preds),
    'Accuracy': accuracy_score(actual, preds),
    'Recall': recall_score(actual, preds),
    'Precision': precision_score(actual, preds)
})

3/3 [==============================] - 0s 1ms/step


,activation_function,epochs,learning_rate,F1 Score,Accuracy,Recall,Precision
0,sigmoid,10,0.1,0.777778,0.688312,0.84,0.724138


In [19]:
TN, FP, FN, TP = confusion_matrix(y_test, preds).ravel()
# formulas for sens and spec
sensitivity = TP / (TP + FN)  
specificity = TN / (TN + FP)
print(f"Sensitivity is : {sensitivity:.2f}")
print(f"Specificity is : {specificity:.2f}")

Sensitivity is : 0.84
Specificity is : 0.41


In [20]:
grid.best_params_

{'activation': 'sigmoid', 'epochs': 10, 'learning_rate': 0.1}

In above example the first model is overfitting and the last two models are underfitting, so the best model is model w for sigmoid , 0.01 learning rate and 10 epochs

CHANGING EPOCHS EXPERIMENT 2

In [74]:
model_epochs = Perceptron_algorithm(learning_rate=0.1, activation='sigmoid')
model_epochs.fit(X_train, y_train, epochs = 2000)
preds = model_epochs.predict(X_test)

preds_bin = []
for i in preds:
    if i < 0.5:
        preds_bin.append(0)
    else:
        preds_bin.append(1)
preds_bin = np.array(preds_bin)

print("Test Accuracy score: ", accuracy_score(y_test, preds_bin))

Epoch 1/2000
22/22 [==============================] - 0s 521us/step - loss: 0.5847 - accuracy: 0.7164
Epoch 2/2000
22/22 [==============================] - 0s 432us/step - loss: 0.5739 - accuracy: 0.7091
Epoch 3/2000
22/22 [==============================] - 0s 504us/step - loss: 0.5657 - accuracy: 0.7091
Epoch 4/2000
22/22 [==============================] - 0s 531us/step - loss: 0.5593 - accuracy: 0.7178
Epoch 5/2000
22/22 [==============================] - 0s 480us/step - loss: 0.5527 - accuracy: 0.7221
Epoch 6/2000
22/22 [==============================] - 0s 433us/step - loss: 0.5472 - accuracy: 0.7250
Epoch 7/2000
22/22 [==============================] - 0s 432us/step - loss: 0.5423 - accuracy: 0.7250
Epoch 8/2000
22/22 [==============================] - 0s 432us/step - loss: 0.5383 - accuracy: 0.7352
Epoch 9/2000
22/22 [==============================] - 0s 429us/step - loss: 0.5341 - accuracy: 0.7294
Epoch 10/2000
22/22 [==============================] - 0s 429us/step - loss: 0.530

In [75]:
TN, FP, FN, TP = confusion_matrix(y_test, preds_bin).ravel()
# formulas for sens and spec
sensitivity = TP / (TP + FN)  
specificity = TN / (TN + FP)
print(f"Sensitivity is : {sensitivity:.2f}")
print(f"Specificity is : {specificity:.2f}")

Sensitivity is : 0.90
Specificity is : 0.52


In [77]:
preds = model_epochs.predict(X_train)
preds_bin = []
for i in preds:
    if i < 0.5:
        preds_bin.append(0)
    else:
        preds_bin.append(1)
preds_bin = np.array(preds_bin)
print("--------------------------------------------------------------")
print()
print("Training data parameters")
print()
print("--------------------------------------------------------------")
print("Accuracy score: ", accuracy_score(y_train, preds_bin))

22/22 [==============================] - 0s 338us/step
--------------------------------------------------------------

Training data parameters

--------------------------------------------------------------
Accuracy score:  0.784370477568741


500 epochs

In [55]:
model_epochs = Perceptron_algorithm(learning_rate=0.1, activation='sigmoid')
model_epochs.fit(X_train, y_train, epochs = 500)
preds = model_epochs.predict(X_test)

preds_bin = []
for i in preds:
    if i < 0.5:
        preds_bin.append(0)
    else:
        preds_bin.append(1)
preds_bin = np.array(preds_bin)

print("Test Accuracy score: ", accuracy_score(y_test, preds_bin)) # Test accuracy

Epoch 1/500
22/22 [==============================] - 0s 483us/step - loss: 0.5992 - accuracy: 0.6657
Epoch 2/500
22/22 [==============================] - 0s 538us/step - loss: 0.5862 - accuracy: 0.6860
Epoch 3/500
22/22 [==============================] - 0s 528us/step - loss: 0.5753 - accuracy: 0.6932
Epoch 4/500
22/22 [==============================] - 0s 505us/step - loss: 0.5652 - accuracy: 0.7004
Epoch 5/500
22/22 [==============================] - 0s 528us/step - loss: 0.5571 - accuracy: 0.7091
Epoch 6/500
22/22 [==============================] - 0s 475us/step - loss: 0.5495 - accuracy: 0.7149
Epoch 7/500
22/22 [==============================] - 0s 432us/step - loss: 0.5428 - accuracy: 0.7135
Epoch 8/500
22/22 [==============================] - 0s 480us/step - loss: 0.5368 - accuracy: 0.7192
Epoch 9/500
22/22 [==============================] - 0s 476us/step - loss: 0.5315 - accuracy: 0.7250
Epoch 10/500
22/22 [==============================] - 0s 432us/step - loss: 0.5266 - accura

In [56]:
TN, FP, FN, TP = confusion_matrix(y_test, preds_bin).ravel()
# formulas for sens and spec
sensitivity = TP / (TP + FN)  
specificity = TN / (TN + FP)
print(f"Sensitivity is : {sensitivity:.2f}")
print(f"Specificity is : {specificity:.2f}")

Sensitivity is : 0.90
Specificity is : 0.52


In [57]:
preds = model_epochs.predict(X_train)
preds_bin = []
for i in preds:
    if i < 0.5:
        preds_bin.append(0)
    else:
        preds_bin.append(1)
preds_bin = np.array(preds_bin)
print("--------------------------------------------------------------")
print()
print("Training data parameters")
print()
print("--------------------------------------------------------------")
print("Accuracy score: ", accuracy_score(y_train, preds_bin))

 1/22 [>.............................] - ETA: 0s

22/22 [==============================] - 0s 428us/step
--------------------------------------------------------------

Training data parameters

--------------------------------------------------------------
Accuracy score:  0.7814761215629522


In [62]:
model_epochs = Perceptron_algorithm(learning_rate=0.1, activation='sigmoid')
model_epochs.fit(X_train, y_train, epochs = 100)
preds = model_epochs.predict(X_test)

preds_bin = []
for i in preds:
    if i < 0.5:
        preds_bin.append(0)
    else:
        preds_bin.append(1)
preds_bin = np.array(preds_bin)

print("--------------------------------------------------------------")
print()
print("Test Accuracy score: ", accuracy_score(y_test, preds_bin))
print()
print("--------------------------------------------------------------")

Epoch 1/100
22/22 [==============================] - 0s 542us/step - loss: 0.6641 - accuracy: 0.6498
Epoch 2/100
22/22 [==============================] - 0s 527us/step - loss: 0.6392 - accuracy: 0.6614
Epoch 3/100
22/22 [==============================] - 0s 456us/step - loss: 0.6212 - accuracy: 0.6686
Epoch 4/100
22/22 [==============================] - 0s 481us/step - loss: 0.6068 - accuracy: 0.6700
Epoch 5/100
22/22 [==============================] - 0s 476us/step - loss: 0.5943 - accuracy: 0.6802
Epoch 6/100
22/22 [==============================] - 0s 483us/step - loss: 0.5833 - accuracy: 0.6874
Epoch 7/100
22/22 [==============================] - 0s 433us/step - loss: 0.5730 - accuracy: 0.6932
Epoch 8/100
22/22 [==============================] - 0s 479us/step - loss: 0.5648 - accuracy: 0.7004
Epoch 9/100
22/22 [==============================] - 0s 429us/step - loss: 0.5568 - accuracy: 0.7004
Epoch 10/100
22/22 [==============================] - 0s 384us/step - loss: 0.5499 - accura

In [60]:
TN, FP, FN, TP = confusion_matrix(y_test, preds_bin).ravel()
# formulas for sens and spec
sensitivity = TP / (TP + FN)  
specificity = TN / (TN + FP)

print(f"Sensitivity is : {sensitivity:.2f}")
print(f"Specificity is : {specificity:.2f}")

Sensitivity is : 0.88
Specificity is : 0.52


In [61]:
preds = model_epochs.predict(X_train)
preds_bin = []
for i in preds:
    if i < 0.5:
        preds_bin.append(0)
    else:
        preds_bin.append(1)
preds_bin = np.array(preds_bin)

print("--------------------------------------------------------------")
print()
print("Training data parameters")
print()
print("--------------------------------------------------------------")
print("Accuracy score: ", accuracy_score(y_train, preds_bin))

22/22 [==============================] - 0s 334us/step
--------------------------------------------------------------

Training data parameters

--------------------------------------------------------------
Accuracy score:  0.7756874095513748


The value above is predicted accuracy for test score

30 Epochs

In [63]:
model_epochs = Perceptron_algorithm(learning_rate=0.1, activation='sigmoid')
model_epochs.fit(X_train, y_train, epochs = 30)
preds = model_epochs.predict(X_test)

preds_bin = []
for i in preds:
    if i < 0.5:
        preds_bin.append(0)
    else:
        preds_bin.append(1)
# preds_bin = np.array(preds_bin)

print("--------------------------------------------------------------")
print()
print("Testing data parameters")
print()
print("--------------------------------------------------------------")
print("Accuracy score: ", accuracy_score(y_test, preds_bin))

Epoch 1/30
22/22 [==============================] - 0s 491us/step - loss: 0.6270 - accuracy: 0.6585
Epoch 2/30
22/22 [==============================] - 0s 485us/step - loss: 0.6132 - accuracy: 0.6628
Epoch 3/30
22/22 [==============================] - 0s 504us/step - loss: 0.6015 - accuracy: 0.6700
Epoch 4/30
22/22 [==============================] - 0s 482us/step - loss: 0.5926 - accuracy: 0.6729
Epoch 5/30
22/22 [==============================] - 0s 480us/step - loss: 0.5834 - accuracy: 0.6860
Epoch 6/30
22/22 [==============================] - 0s 381us/step - loss: 0.5755 - accuracy: 0.6889
Epoch 7/30
22/22 [==============================] - 0s 432us/step - loss: 0.5679 - accuracy: 0.6961
Epoch 8/30
22/22 [==============================] - 0s 384us/step - loss: 0.5606 - accuracy: 0.6961
Epoch 9/30
22/22 [==============================] - 0s 432us/step - loss: 0.5546 - accuracy: 0.6990
Epoch 10/30
22/22 [==============================] - 0s 479us/step - loss: 0.5495 - accuracy: 0.7091

In [25]:
TN, FP, FN, TP = confusion_matrix(y_test, preds_bin).ravel()
# formulas for sens and spec
sensitivity = TP / (TP + FN)  
specificity = TN / (TN + FP)
print(f"Sensitivity is : {sensitivity:.2f}")
print(f"Specificity is : {specificity:.2f}")

Sensitivity is : 0.88
Specificity is : 0.48


The value above is predicted accuracy for test score

In [26]:
preds = model_epochs.predict(X_train)
preds_bin = []
for i in preds:
    if i < 0.5:
        preds_bin.append(0)
    else:
        preds_bin.append(1)


print("--------------------------------------------------------------")
print()
print("Training data parameters")
print()
print("--------------------------------------------------------------")
print("Accuracy score: ", accuracy_score(y_train, preds_bin))

22/22 [==============================] - 0s 333us/step
--------------------------------------------------------------

Training data parameters

--------------------------------------------------------------
Accuracy score:  0.7670043415340086


The value above is predicted accuracy for train score

In [91]:
import matplotlib.pyplot as plt

# Data from the models dictionary
models = {
    "model_0" : {
        "number of epochs": 2000,
        "test_accu": 0.7662,
        "sens": 0.90,
        "spec": 0.52,
        "train_accu": 0.7843
    },

    "model_1": {
        "number of epochs": 500,
        "test_accu": 0.7662,
        "sens": 0.90,
        "spec": 0.52,
        "train_accu": 0.7815
    },
    "model_2": {
        "number of epochs": 100,
        "test_accu": 0.7532,
        "sens": 0.88,
        "spec": 0.52,
        "train_accu": 0.7757
    },
    "model_3": {
        "number of epochs": 30,
        "test_accu": 0.7273,
        "sens": 0.88,
        "spec": 0.48,
        "train_accu": 0.7670
    }
}

# Making a results dataframe
results_dataframe_exp_2 = pd.DataFrame.from_dict(models, orient='index')
results_dataframe_exp_2.reset_index(inplace=True)
results_dataframe_exp_2.rename(columns={'index': 'Model'}, inplace=True)
results_dataframe_exp_2

,Model,number of epochs,test_accu,sens,spec,train_accu
0,model_0,2000,0.7662,0.90,0.52,0.7843
1,model_1,500,0.7662,0.90,0.52,0.7815
2,model_2,100,0.7532,0.88,0.52,0.7757
3,model_3,30,0.7273,0.88,0.48,0.7670


CHANGING LEARNING RATES

In [92]:
model_epochs = Perceptron_algorithm(learning_rate=0.01, activation='sigmoid')
model_epochs.fit(X_train, y_train, epochs = 500)
preds = model_epochs.predict(X_test)

preds_bin = []
for i in preds:
    if i < 0.5:
        preds_bin.append(0)
    else:
        preds_bin.append(1)
# preds_bin = np.array(preds_bin)

print("--------------------------------------------------------------")
print()
print("Testing data parameters")
print()
print("--------------------------------------------------------------")
print("Accuracy score: ", accuracy_score(y_test, preds_bin))

Epoch 1/500
22/22 [==============================] - 0s 528us/step - loss: 0.6660 - accuracy: 0.6556
Epoch 2/500
22/22 [==============================] - 0s 579us/step - loss: 0.6475 - accuracy: 0.6816
Epoch 3/500
22/22 [==============================] - 0s 476us/step - loss: 0.6342 - accuracy: 0.6874
Epoch 4/500
22/22 [==============================] - 0s 480us/step - loss: 0.6244 - accuracy: 0.6860
Epoch 5/500
22/22 [==============================] - 0s 481us/step - loss: 0.6172 - accuracy: 0.6802
Epoch 6/500
22/22 [==============================] - 0s 480us/step - loss: 0.6117 - accuracy: 0.6758
Epoch 7/500
22/22 [==============================] - 0s 429us/step - loss: 0.6074 - accuracy: 0.6787
Epoch 8/500
22/22 [==============================] - 0s 476us/step - loss: 0.6042 - accuracy: 0.6773
Epoch 9/500
22/22 [==============================] - 0s 481us/step - loss: 0.6015 - accuracy: 0.6787
Epoch 10/500
22/22 [==============================] - 0s 428us/step - loss: 0.5993 - accura

In [94]:
TN, FP, FN, TP = confusion_matrix(y_test, preds_bin).ravel()
# formulas for sens and spec
sensitivity = TP / (TP + FN)  
specificity = TN / (TN + FP)
print(f"Sensitivity is : {sensitivity:.2f}")
print(f"Specificity is : {specificity:.2f}")

preds = model_epochs.predict(X_train)
preds_bin = []
for i in preds:
    if i < 0.5:
        preds_bin.append(0)
    else:
        preds_bin.append(1)


print("--------------------------------------------------------------")
print()
print("Training data parameters")
print()
print("--------------------------------------------------------------")
print("Accuracy score: ", accuracy_score(y_train, preds_bin))

Sensitivity is : 0.88
Specificity is : 0.48
22/22 [==============================] - 0s 341us/step
--------------------------------------------------------------

Training data parameters

--------------------------------------------------------------
Accuracy score:  0.7785817655571635


In [95]:
model_epochs = Perceptron_algorithm(learning_rate=0.001, activation='sigmoid')
model_epochs.fit(X_train, y_train, epochs = 500)
preds = model_epochs.predict(X_test)

preds_bin = []
for i in preds:
    if i < 0.5:
        preds_bin.append(0)
    else:
        preds_bin.append(1)
# preds_bin = np.array(preds_bin)

print("--------------------------------------------------------------")
print()
print("Testing data parameters")
print()
print("--------------------------------------------------------------")
print("Accuracy score: ", accuracy_score(y_test, preds_bin))

Epoch 1/500
22/22 [==============================] - 0s 524us/step - loss: 0.6170 - accuracy: 0.6932
Epoch 2/500
22/22 [==============================] - 0s 506us/step - loss: 0.6163 - accuracy: 0.6903
Epoch 3/500
22/22 [==============================] - 0s 429us/step - loss: 0.6157 - accuracy: 0.6903
Epoch 4/500
22/22 [==============================] - 0s 480us/step - loss: 0.6151 - accuracy: 0.6860
Epoch 5/500
22/22 [==============================] - 0s 429us/step - loss: 0.6145 - accuracy: 0.6860
Epoch 6/500
22/22 [==============================] - 0s 461us/step - loss: 0.6139 - accuracy: 0.6874
Epoch 7/500
22/22 [==============================] - 0s 476us/step - loss: 0.6134 - accuracy: 0.6874
Epoch 8/500
22/22 [==============================] - 0s 480us/step - loss: 0.6128 - accuracy: 0.6889
Epoch 9/500
22/22 [==============================] - 0s 429us/step - loss: 0.6123 - accuracy: 0.6860
Epoch 10/500
22/22 [==============================] - 0s 454us/step - loss: 0.6118 - accura

In [96]:
TN, FP, FN, TP = confusion_matrix(y_test, preds_bin).ravel()
# formulas for sens and spec
sensitivity = TP / (TP + FN)  
specificity = TN / (TN + FP)
print(f"Sensitivity is : {sensitivity:.2f}")
print(f"Specificity is : {specificity:.2f}")

preds = model_epochs.predict(X_train)
preds_bin = []
for i in preds:
    if i < 0.5:
        preds_bin.append(0)
    else:
        preds_bin.append(1)


print("--------------------------------------------------------------")
print()
print("Training data parameters")
print()
print("--------------------------------------------------------------")
print("Accuracy score: ", accuracy_score(y_train, preds_bin))

Sensitivity is : 0.84
Specificity is : 0.30
22/22 [==============================] - 0s 381us/step
--------------------------------------------------------------

Training data parameters

--------------------------------------------------------------
Accuracy score:  0.7105643994211288


MULTI-LAYER PERCEPTRON

In [50]:
model = keras.Sequential()
# This is hidden layer 1 
model.add(layers.Dense(150, activation='tanh')) 
# This is hidden layer 2 
model.add(layers.Dense(100, activation='sigmoid')) 
# This is hidden layer 3
model.add(layers.Dense(50, activation='tanh')) 
# This is output layer
model.add(layers.Dense(1, activation='sigmoid')) 

# specify the Adam optimizer and after this compliling and fitting on the trainin dataset
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=150, batch_size=32)

Epoch 1/150
22/22 [==============================] - 0s 1ms/step - loss: 0.6389 - accuracy: 0.6483
Epoch 2/150
22/22 [==============================] - 0s 910us/step - loss: 0.5720 - accuracy: 0.6932
Epoch 3/150
22/22 [==============================] - 0s 864us/step - loss: 0.5172 - accuracy: 0.7467
Epoch 4/150
22/22 [==============================] - 0s 862us/step - loss: 0.4910 - accuracy: 0.7583
Epoch 5/150
22/22 [==============================] - 0s 866us/step - loss: 0.4759 - accuracy: 0.7728
Epoch 6/150
22/22 [==============================] - 0s 877us/step - loss: 0.4852 - accuracy: 0.7757
Epoch 7/150
22/22 [==============================] - 0s 909us/step - loss: 0.4772 - accuracy: 0.7800
Epoch 8/150
22/22 [==============================] - 0s 952us/step - loss: 0.4745 - accuracy: 0.7713
Epoch 9/150
22/22 [==============================] - 0s 914us/step - loss: 0.4738 - accuracy: 0.7713
Epoch 10/150
22/22 [==============================] - 0s 862us/step - loss: 0.4751 - accuracy

In [97]:
pred_0 = model.predict(X_train)
bp = (pred_0 > 0.5).astype(int)
loss, accuracy = model.evaluate(X_train, y_train)
print('Training accuracy score')
print("Accuracy score: ", accuracy_score(y_train, bp))

22/22 [==============================] - 0s 857us/step - loss: 0.4341 - accuracy: 0.7931
Accuracy score:  0.7930535455861071


In [52]:
pred_1 = model.predict(X_test)
bp = (pred_1 > 0.5).astype(int)
print('Testing accuacy score')
print("Accuracy score: ", accuracy_score(y_test, bp))

3/3 [==============================] - 0s 1ms/step
Accuracy score:  0.7792207792207793


In [53]:
TN, FP, FN, TP = confusion_matrix(y_test, bp).ravel()
# formulas for sens and spec
sensitivity = TP / (TP + FN)  
specificity = TN / (TN + FP)
print(f"Sensitivity is : {sensitivity:.2f}")
print(f"Specificity is : {specificity:.2f}")

Sensitivity is : 0.92
Specificity is : 0.52


In [31]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
train_loss

0.43045949935913086